# 멀티모달 RAG 챗봇 프로젝트 설정

AWS Bedrock과 Knowledge Base를 활용한 멀티모달 RAG 챗봇 프로젝트의 개발 환경을 설정합니다.

## 📋 설정 단계
1. 가상환경 생성 및 활성화
2. 필요한 패키지 설치
3. 주피터 커널 등록
4. 환경 변수 설정
5. AWS 자격 증명 확인
6. 설치 확인

## 1. 가상환경 생성 및 활성화

**터미널에서 실행해주세요:**

In [ ]:
# 가상환경 생성
!python -m venv my-rag-venv
!source my-rag-venv/bin/activate

SyntaxError: invalid syntax (1225417612.py, line 3)

In [ ]:
#requirements.txt 파일 설치
pip install --upgrade pip
pip install -r requirements.txt

In [ ]:
# 터미널에서 실행할 명령어들 (참고용)


commands = [
    "# 1. 가상환경 생성",
    "python -m venv my-rag-venv",
    "",
    "# 2. 가상환경 활성화",
    "source my-rag-venv/bin/activate  # Linux/Mac",
    "# my-rag-venv\\Scripts\\activate  # Windows",
    "",
    "# 3. pip 업그레이드",
    "pip install --upgrade pip"
]

for cmd in commands:
    print(cmd)

## 3. Requirements 설치

**가상환경이 활성화된 터미널에서 실행:**

In [ ]:
# 터미널에서 실행: pip install -r requirements.txt
print("터미널에서 실행하세요: pip install -r requirements.txt")
print("\n설치될 주요 패키지들:")

packages = [
    "streamlit>=1.28.0 - 웹 UI 프레임워크",
    "boto3>=1.34.0 - AWS SDK",
    "python-dotenv>=1.0.0 - 환경 변수 관리",
    "pdfplumber==0.10.3 - PDF 텍스트 추출",
    "pypdf==4.0.1 - PDF 처리",
    "unstructured - 멀티모달 문서 처리",
    "Pillow>=10.0.0 - 이미지 처리",
    "opencv-python==4.9.0.80 - 이미지 전처리",
    "pytesseract>=0.3.10 - OCR",
    "opensearch-py==2.4.2 - OpenSearch 클라이언트"
]

for pkg in packages:
    print(f"  - {pkg}")

## 4. 환경 변수 설정

`.env` 파일을 생성하고 필요한 환경 변수를 설정합니다.

In [ ]:
# .env 파일 템플릿 생성
env_template = """
# AWS 설정
AWS_REGION=us-east-1
AWS_PROFILE=default

# Bedrock 설정
BEDROCK_KNOWLEDGE_BASE_ID=your-knowledge-base-id
BEDROCK_MODEL_ID=anthropic.claude-3-7-sonnet-20240620-v1:0
BEDROCK_EMBEDDING_MODEL_ID=amazon.titan-embed-text-v2:0
BEDROCK_RERANK_MODEL_ID=cohere.rerank-v3-5:0

# S3 설정
S3_BUCKET_NAME=your-s3-bucket-name
S3_PREFIX=documents/

# 애플리케이션 설정
MAX_FILE_SIZE_MB=10
SUPPORTED_FILE_TYPES=pdf,txt,md,jpg,jpeg,png
"""

# .env 파일 생성
with open('.env.example', 'w') as f:
    f.write(env_template.strip())

print(".env.example 파일이 생성되었습니다.")
print("\n다음 단계:")
print("1. .env.example을 .env로 복사")
print("2. .env 파일에서 실제 값들로 수정")
print("\n터미널에서 실행:")
print("cp .env.example .env")

## 5. AWS 자격 증명 확인

In [ ]:
import boto3
from botocore.exceptions import NoCredentialsError, ClientError

def check_aws_credentials():
    """AWS 자격 증명 확인"""
    try:
        # STS 클라이언트로 자격 증명 확인
        sts = boto3.client('sts')
        identity = sts.get_caller_identity()
        
        print("✅ AWS 자격 증명 확인 성공!")
        print(f"Account ID: {identity.get('Account')}")
        print(f"User ARN: {identity.get('Arn')}")
        print(f"User ID: {identity.get('UserId')}")
        
        return True
        
    except NoCredentialsError:
        print("❌ AWS 자격 증명이 설정되지 않았습니다.")
        print("다음 명령어로 설정하세요: aws configure")
        return False
        
    except ClientError as e:
        print(f"❌ AWS 자격 증명 오류: {e}")
        return False
    except Exception as e:
        print(f"❌ 예상치 못한 오류: {e}")
        return False

# AWS 자격 증명 확인 실행
check_aws_credentials()

## 6. Bedrock 서비스 접근 확인

In [ ]:
def check_bedrock_access():
    """Bedrock 서비스 접근 확인"""
    try:
        # Bedrock 클라이언트 생성
        bedrock = boto3.client('bedrock', region_name='us-east-1')
        
        # Foundation Models 목록 조회
        response = bedrock.list_foundation_models()
        
        print("✅ Bedrock 서비스 접근 성공!")
        print(f"사용 가능한 Foundation Models: {len(response['modelSummaries'])}개")
        
        # Claude 모델 확인
        claude_models = [model for model in response['modelSummaries'] 
                        if 'claude' in model['modelId'].lower()]
        
        if claude_models:
            print("\n사용 가능한 Claude 모델들:")
            for model in claude_models[:3]:  # 처음 3개만 표시
                print(f"  - {model['modelId']}")
        
        return True
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'AccessDeniedException':
            print("❌ Bedrock 서비스 접근 권한이 없습니다.")
            print("IAM 정책에서 Bedrock 권한을 확인하세요.")
        else:
            print(f"❌ Bedrock 접근 오류: {e}")
        return False
        
    except Exception as e:
        print(f"❌ 예상치 못한 오류: {e}")
        return False

# Bedrock 접근 확인 실행
check_bedrock_access()

## 7. 설치된 패키지 확인

In [ ]:
import pkg_resources
import sys

def check_installed_packages():
    """주요 패키지 설치 확인"""
    required_packages = [
        'streamlit',
        'boto3',
        'python-dotenv',
        'pdfplumber',
        'pypdf',
        'Pillow',
        'opencv-python',
        'pytesseract',
        'opensearch-py'
    ]
    
    print("📦 주요 패키지 설치 확인:")
    print("=" * 50)
    
    for package in required_packages:
        try:
            version = pkg_resources.get_distribution(package).version
            print(f"✅ {package:<20} {version}")
        except pkg_resources.DistributionNotFound:
            print(f"❌ {package:<20} 설치되지 않음")
    
    print("\n🐍 Python 정보:")
    print(f"Python 버전: {sys.version}")
    print(f"Python 경로: {sys.executable}")

# 패키지 확인 실행
check_installed_packages()

## 8. 프로젝트 구조 생성

In [ ]:
import os

def create_project_structure():
    """프로젝트 디렉토리 구조 생성"""
    directories = [
        'src',
        'config',
        'data/uploads',
        'data/samples',
        'tests'
    ]
    
    files = [
        'src/__init__.py',
        'src/bedrock_client.py',
        'src/knowledge_base.py',
        'src/document_processor.py',
        'src/chat_handler.py',
        'src/utils.py',
        'config/__init__.py',
        'config/settings.py',
        'tests/__init__.py',
        '.gitignore'
    ]
    
    print("📁 프로젝트 구조 생성:")
    print("=" * 30)
    
    # 디렉토리 생성
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
        print(f"✅ 디렉토리 생성: {directory}/")
    
    # 빈 파일 생성
    for file_path in files:
        if not os.path.exists(file_path):
            with open(file_path, 'w') as f:
                if file_path.endswith('.py'):
                    f.write('# TODO: 구현 예정\n')
                elif file_path == '.gitignore':
                    f.write('''# Python
__pycache__/
*.py[cod]
*.so
.Python
env/
venv/
my-rag-venv/

# Environment variables
.env

# Jupyter Notebook
.ipynb_checkpoints

# Data
data/uploads/*
!data/uploads/.gitkeep

# Logs
*.log

# IDE
.vscode/
.idea/
''')
            print(f"✅ 파일 생성: {file_path}")
        else:
            print(f"⚠️  파일 존재: {file_path}")
    
    # .gitkeep 파일 생성
    gitkeep_files = ['data/uploads/.gitkeep', 'data/samples/.gitkeep']
    for gitkeep in gitkeep_files:
        with open(gitkeep, 'w') as f:
            f.write('')
        print(f"✅ 파일 생성: {gitkeep}")

# 프로젝트 구조 생성 실행
create_project_structure()

## 9. 설정 완료 확인

In [ ]:
def setup_completion_check():
    """설정 완료 상태 확인"""
    print("🎉 프로젝트 설정 완료 확인")
    print("=" * 40)
    
    checks = [
        ("가상환경 활성화", "커널이 'My RAG Chatbot'으로 설정되어 있는지 확인"),
        ("패키지 설치", "위의 패키지 확인에서 모든 패키지가 설치되었는지 확인"),
        ("AWS 자격 증명", "AWS 자격 증명이 정상적으로 설정되었는지 확인"),
        ("Bedrock 접근", "Bedrock 서비스에 접근 가능한지 확인"),
        ("환경 변수", ".env 파일을 생성하고 실제 값으로 수정했는지 확인"),
        ("프로젝트 구조", "필요한 디렉토리와 파일들이 생성되었는지 확인")
    ]
    
    for i, (item, description) in enumerate(checks, 1):
        print(f"{i}. {item}: {description}")
    
    print("\n✨ 다음 단계:")
    print("1. AWS Bedrock Knowledge Base 생성")
    print("2. S3 버킷 설정")
    print("3. 기본 코드 구현 시작")
    print("4. Streamlit 앱 개발")

# 설정 완료 확인 실행
setup_completion_check()

## 📝 참고사항

### 주피터 노트북 커널 변경 방법
1. 노트북 상단의 "Kernel" 메뉴 클릭
2. "Change kernel" 선택
3. "My RAG Chatbot" 커널 선택

### 문제 해결
- **패키지 설치 오류**: `pip install --upgrade pip` 후 재시도
- **AWS 자격 증명 오류**: `aws configure` 명령어로 재설정
- **Bedrock 접근 오류**: IAM 정책에서 Bedrock 권한 확인

### 유용한 명령어
```bash
# 가상환경 비활성화
deactivate

# 설치된 패키지 목록 저장
pip freeze > requirements_freeze.txt

# 주피터 커널 목록 확인
jupyter kernelspec list

# 커널 삭제 (필요시)
jupyter kernelspec uninstall my-rag-venv
```